In [2]:
# Import necessary packages
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from urllib.request import urlopen
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [3]:
list_of_genres = ['children', 'fantasy', 'magical', 'mystery', 'action', 'crime', 'drama', 'science fiction', 'adventure', 'biography', 'comedy', 'thriller', 'musical', 'western', 'romance', 'animation', 'war', 'horror', 'sport', 'political', 'saga', ' satire', 'social']

In [4]:
#Works
def get_IMDB_names(url):
    rn = requests.get(url)
    soup = BeautifulSoup(rn.text, 'html.parser')
    movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
    names = []
    for container in movie_containers:
# Extract:
# The name
        name = container.select('img')[0]['alt']
        names.append(name)
    return names

In [5]:
def get_IMDB_years(url):
    ry = requests.get(url)
    soup = BeautifulSoup(ry.text, 'html.parser')
    movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
    years = []
    for container in movie_containers:
        # The year
        year = container.find('span', class_ = 'lister-item-year').text
        year = str(year)
        years.append(year)
        year = str(year)
        year = year[1:5]
    return years

In [6]:
def get_IMDB_genres(url):
    rg = requests.get(url)
    soup = BeautifulSoup(rg.text, 'html.parser')
    movie_containers = soup.find_all('div', class_ = 'lister-item mode-advanced')
    genres =[] 
    for container in movie_containers:
        genre_container = container.find('p')
        genre = genre_container.find('span', class_='genre')
        #genre = genre.text
        #genre = str(genre)
        #genre = genre[1:]
        genres.append(genre)
    return genres

In [9]:
#function that will get the violence grade for a year
def get_IMDB_table(year):
    url = 'https://www.imdb.com/search/title/?release_date='+ str(year) + '-01-01,' + str(year) + '-12-31&user_rating=1.0,10.0&countries=us&sort=user_rating,desc&count=250'
    names = get_IMDB_names(url)
    #years = get_IMDB_years(url)
    genres = get_IMDB_genres(url)
    initial_df = pd.DataFrame({'name': names, 'genre': genres})
    #turn BS elements into strings
    initial_df['genre']=initial_df['genre'].map(lambda x: str(x))
    # add a year column             
    years = [year]*len(initial_df)
    initial_df['year'] = years
    #create a list with all the genres from the "Genres" column and lower the case of each element
    list_year_1 = initial_df['genre'].to_list()
    lower_list = [x.lower() for x in list_year_1]
    #separate the genres where a movie has two or more genres
    list_final = [x.split() for x in lower_list]
    list_year = []
    for sublist in list_final:
        for item in sublist:
            if item == '<span':
                continue
            elif item == 'class="genre">':
                continue
            elif item == '</span>':
                continue
            else:
                list_year.append(item)
    #some genres have comma attached, take off comma
            list_year_no_comma = []
            for item in list_year:
                if ',' in item:
                        list_year_no_comma.append(item[:-1])
                else:
                            list_year_no_comma.append(item)
    #return list_year_no_comma
    # Create a dictionary key- genres, value = number of movies of a certain genre
    genre_counts = {}
# Iterate through genre list
    for genre in list_year_no_comma:
# Update word counts - keep only genres in the general list of genres
        if genre in list_of_genres:
            genre_counts[genre] = genre_counts.get(genre, 0) + 1 #Get previous entry, update by 1
        else:
            continue
#Set 'Genres' as df index
    df_to_plot = pd.DataFrame(genre_counts.items(), columns=['Genres', 'Genre_count'])
    #return df_to_plot
    df_to_plot.set_index('Genres')
# Join the the 2 Dataframes on 'Genres' column and replace NANs with mean value.
    #Zeitgeist_df = war_grades_df.join(df_to_plot.set_index('Genres'), on='Genres')
    #Zeitgeist_df = Zeitgeist_df.fillna(5)
#Create a column year_war_grade 
    #Zeitgeist_df['year_war_grade'] = Zeitgeist_df['Grade']*Zeitgeist_df['Year']
# Add a year column
    year = year
    years_final = [year]*len(df_to_plot)
    df_to_plot['Year'] = years_final
    #del Zeitgeist_df['Grade']
           
    return df_to_plot
#Zeitgeist1980 = get_table_after_1996(1980)
#Zeitgeist1980
get_IMDB_table(1980)

,Genres,Genre_count,Year
0,thriller,2,1980
1,horror,2,1980
2,adventure,27,1980
3,drama,87,1980
4,comedy,108,1980
5,action,18,1980
6,crime,7,1980
7,mystery,3,1980
8,sport,3,1980
9,romance,16,1980


In [11]:
pieces = []
for year in [*range(1980, 2020,1)]:
    df_year = get_IMDB_table(year)
    pieces.append(df_year)
df_all = pd.concat(pieces)
df_all
df_all.to_csv('IMDB_genres_1980_2019.csv', index=False, header= True)    


In [ ]:
grouped = df_all.groupby(['Year'])['year_war_grade'].sum()
grouped.to_csv('grouped_war_grade_IMDB.csv', index='Year', header=True)